In [1]:
import pdfplumber
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

In [2]:
import pdfplumber
import re

def extract_paragraphs_from_pdf(pdf_path):
    paragraphs = []

    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            paragraph_list = re.split(r'(?<=\d):', text)
            paragraph_list = [paragraph.strip() for paragraph in paragraph_list if paragraph.strip()]
            paragraphs.extend(paragraph_list)

    return paragraphs


pdf_path = 'C:/Users/sriyo/Desktop/jets/NLP Bot/files/NCERT Biology.pdf'
all_paragraphs = extract_paragraphs_from_pdf(pdf_path)

for i, paragraph in enumerate(all_paragraphs, start=1):
    print(f"Paragraph {i}: {paragraph}")


Paragraph 1: CHAPTER 2
HUMAN REPRODUCTION
2.1 The Male Reproductive
System
As you are aware, humans are sexually reproducing and
2.2 The Female Reproductive
viviparous. The reproductive events in humans include
System
formation of gametes (gametogenesis), i.e., sperms in males
2.3 Gametogenesis
and ovum in females, transfer of sperms into the female
2.4 Menstrual Cycle genital tract (insemination) and fusion of male and female
gametes (fertilisation) leading to formation of zygote. This
2.5 Fertilisation and
is followed by formation and development of blastocyst
Implantation
and its attachment to the uterine wall (implantation),
2.6 Pregnancy and Embryonic embryonic development (gestation) and delivery of the
Development baby (parturition). You have learnt that these reproductive
events occur after puberty. There are remarkable
2.7 Parturition and Lactation
differences between the reproductive events in the male
and in the female, for example, sperm formation continues
even in old men,

In [3]:
import pdfplumber
import re

def preprocess_text(text):
    text = text.replace('\n', ' ')
    text = re.sub(r'[^a-zA-Z0-9\s.]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = text.lower()
    return text

def extract_and_preprocess_text(file_path):
    preprocessed_texts = [] 
    with pdfplumber.open(file_path) as pdf:
        num_pages = len(pdf.pages)

        for page_num in range(num_pages):
            page = pdf.pages[page_num]
            text = page.extract_text()
            preprocessed_text = preprocess_text(text)
            preprocessed_texts.append(preprocessed_text)

            print(f"Page {page_num + 1}:\n{preprocessed_text}\n")

    return preprocessed_texts 


pdf_file_path = 'C:/Users/sriyo/Desktop/jets/NLP Bot/files/NCERT Biology.pdf'
preprocessed_texts = extract_and_preprocess_text(pdf_file_path)



Page 1:
chapter 2 human reproduction 2.1 the male reproductive system as you are aware humans are sexually reproducing and 2.2 the female reproductive viviparous. the reproductive events in humans include system formation of gametes gametogenesis i.e. sperms in males 2.3 gametogenesis and ovum in females transfer of sperms into the female 2.4 menstrual cycle genital tract insemination and fusion of male and female gametes fertilisation leading to formation of zygote. this 2.5 fertilisation and is followed by formation and development of blastocyst implantation and its attachment to the uterine wall implantation 2.6 pregnancy and embryonic embryonic development gestation and delivery of the development baby parturition. you have learnt that these reproductive events occur after puberty. there are remarkable 2.7 parturition and lactation differences between the reproductive events in the male and in the female for example sperm formation continues even in old men but formation of ovum ce

In [4]:
import pdfplumber
import re

def preprocess_text(text):
    text = text.replace('\n', ' ')
    text = re.sub(r'[^a-zA-Z0-9\s.]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = text.lower()
    return text

def clean_and_store_paragraphs(file_path, output_file_path='preprocessed_paragraphs.txt'):
    preprocessed_paragraphs = [] 
    with pdfplumber.open(file_path) as pdf:
        num_pages = len(pdf.pages)

        for page_num in range(num_pages):
            page = pdf.pages[page_num]
            text = page.extract_text()
            preprocessed_text = preprocess_text(text)
       
            paragraphs = re.split(r'(?<=\d):', preprocessed_text)
    
            paragraphs = [paragraph.strip() for paragraph in paragraphs if paragraph.strip()]
            preprocessed_paragraphs.extend(paragraphs)

  
    with open(output_file_path, 'w', encoding='utf-8') as file:
        for paragraph in preprocessed_paragraphs:
            file.write(paragraph + '\n')

    return preprocessed_paragraphs

pdf_file_path = 'C:/Users/sriyo/Desktop/jets/NLP Bot/files/NCERT Biology.pdf'
output_file_path = 'preprocessed_paragraphs.txt'
preprocessed_paragraphs = clean_and_store_paragraphs(pdf_file_path, output_file_path)

# Print the preprocessed paragraphs
for i, paragraph in enumerate(preprocessed_paragraphs, start=1):
    print(f"Preprocessed Paragraph {i}: {paragraph}")

print(f"Preprocessed data saved to: {output_file_path}")


Preprocessed Paragraph 1: chapter 2 human reproduction 2.1 the male reproductive system as you are aware humans are sexually reproducing and 2.2 the female reproductive viviparous. the reproductive events in humans include system formation of gametes gametogenesis i.e. sperms in males 2.3 gametogenesis and ovum in females transfer of sperms into the female 2.4 menstrual cycle genital tract insemination and fusion of male and female gametes fertilisation leading to formation of zygote. this 2.5 fertilisation and is followed by formation and development of blastocyst implantation and its attachment to the uterine wall implantation 2.6 pregnancy and embryonic embryonic development gestation and delivery of the development baby parturition. you have learnt that these reproductive events occur after puberty. there are remarkable 2.7 parturition and lactation differences between the reproductive events in the male and in the female for example sperm formation continues even in old men but fo

In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import re

def generate_questions_for_paragraphs(paragraphs, model, tokenizer, top_k=50, top_p=0.95, num_questions_per_paragraph=2):
    generated_questions = set()

    for paragraph in paragraphs:
        for _ in range(num_questions_per_paragraph):
            input_text_with_prefix = f"Generate a question for the following text: {paragraph}"
            inputs = tokenizer(input_text_with_prefix, return_tensors="pt")

            with torch.no_grad():
                outputs = model.generate(
                    **inputs,
                    max_length=50,
                    top_k=top_k,
                    top_p=top_p,
                    temperature=1.0,
                )

            generated_question = tokenizer.batch_decode(outputs, skip_special_tokens=True)
            split_questions = [question.strip() + '?' for question in re.split(r'[.!?]', generated_question[0])]
            generated_questions.update(split_questions)

    return generated_questions

def save_questions_to_file(questions, output_file_path='generated_questions.txt'):
    with open(output_file_path, 'w', encoding='utf-8') as file:
        for question in questions:
            file.write(question + '\n')

tokenizer = AutoTokenizer.from_pretrained("wiselinjayajos/t5-end2end-questions-generation")
model = AutoModelForSeq2SeqLM.from_pretrained("wiselinjayajos/t5-end2end-questions-generation")


all_generated_questions = generate_questions_for_paragraphs(preprocessed_paragraphs, model, tokenizer, top_k=50, top_p=0.95, num_questions_per_paragraph=2)


save_questions_to_file(all_generated_questions, output_file_path='generated_questions.txt')


print("Generated Questions:")
for question in all_generated_questions:
    print(question)

print(f"Generated questions have been saved to 'generated_questions.txt' and printed.")


c:\Users\sriyo\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\generation\configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


c:\Users\sriyo\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\generation\configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (568 > 512). Running this sequence through the model will result in indexing errors


Generated Questions:
What is the process of release of ovum from a mature follicle called?
What is the hormone secreted?
What is the name?
What is the cavity of the cervix called?
What is the secretions of the male accessory glands?
What is the reproductive cycle in the female primates?
What is the outer layer of the blastomeres called?
What is the male external genitalia?
What is the male reproductive system composed of?
What is the menstrual flow caused by?
What is the male reproductive system located in?
9?
What is the inner layer called?
What is the process called that produces sper?
What is the maximum level of lh during the midcycle called?
What is?
What is the wall of the uterus has three?
What is the female reproductive system located in?
What is the morula?
What are fimbriae?
What is the menstrual phase?
What induces the completion of the meiotic division of the secondary oocyte?
What is the process of formation of mature female gametes called?
What is characteristic of all fe

In [7]:
import language_tool_python

# Load the LanguageTool tool for grammar checking
tool = language_tool_python.LanguageTool('en-US')

def is_grammatically_correct(question):
    matches = tool.check(question)
    return len(matches) == 0

def filter_questions_by_context(questions, context_keywords):
    filtered_questions = []
    for question in questions:
        # Check if the question is grammatically correct
        if is_grammatically_correct(question):
            # Check if the question contains context keywords
            if any(keyword in question.lower() for keyword in context_keywords):
                filtered_questions.append(question)
    return filtered_questions

# Example context keywords related to reproductive biology
context_keywords = ['ovum', 'follicle', 'testis', 'oviduct', 'menstrual', 'female reproductive system']

# Filter questions based on grammar and context
filtered_questions = filter_questions_by_context(all_generated_questions, context_keywords)

# Save the filtered questions to a new text file
save_questions_to_file(filtered_questions, output_file_path='filtered_questions.txt')

# Print the filtered questions
print("Filtered Questions:")
for question in filtered_questions:
    print(question)

print(f"Filtered questions have been saved to 'filtered_questions.txt' and printed.")


Unzipping C:\Users\sriyo\AppData\Local\Temp\tmp20m4fgs_.zip to C:\Users\sriyo\.cache\language_tool_python.
Downloaded https://www.languagetool.org/download/LanguageTool-5.7.zip to C:\Users\sriyo\.cache\language_tool_python.


Filtered Questions:
What is the process of release of ovum from a mature follicle called?
What is the menstrual flow caused by?
What is the female reproductive system located in?
What is the menstrual phase?
How many compartments are in each testis?
What is the menstrual flow?
How long is each testis in adults?
What is the female reproductive system?
What is the last part of the oviduct called?
What is the main event of the menstrual cycle shown in figure 2?
Filtered questions have been saved to 'filtered_questions.txt' and printed.
